# Case study of June 28, 2021

In [4]:
%store  -r city_center clipped_upwind_wedge clipped_downwind_wedge stats

In [5]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [6]:
tracked_cells = xr.open_dataset('C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/stats/trackstats_20210601.0000_20210630.2359.nc')
tracked_cells

<xarray.Dataset>
Dimensions:                  (tracks: 11812, times: 100)
Coordinates:
  * tracks                   (tracks) int64 0 1 2 3 ... 11808 11809 11810 11811
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) int32 ...
    base_time                (tracks, times) datetime64[ns] ...
    meanlat                  (tracks, times) float32 ...
    meanlon                  (tracks, times) float32 ...
    area                     (tracks, times) float32 ...
    cloudnumber              (tracks, times) float64 ...
    ...                       ...
    start_split_tracknumber  (tracks) float64 ...
    start_split_timeindex    (tracks) float64 ...
    start_split_cloudnumber  (tracks) float64 ...
    end_merge_tracknumber    (tracks) float64 ...
    end_merge_timeindex      (tracks) float64 ...
    end_merge_cloudnumber    (tracks) float64 ...
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

## Extract time range

In [7]:
# Define your time range
start_time = np.datetime64('2021-06-28T14:00:00')
end_time = np.datetime64('2021-06-28T19:59:59')

# Create a mask where base_time is within the specified range
time_mask = (tracked_cells.start_basetime >= start_time) & (tracked_cells.start_basetime <= end_time)

# Apply this mask across the 'tracks' dimension
filtered_ds = tracked_cells.where(time_mask, drop=True)
filtered_ds

<xarray.Dataset>
Dimensions:                  (tracks: 276, times: 100)
Coordinates:
  * tracks                   (tracks) int64 9810 9811 9812 ... 10083 10084 10085
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) float64 11.0 1.0 1.0 6.0 ... 3.0 3.0 5.0
    base_time                (tracks, times) datetime64[ns] 2021-06-28T14:01:...
    meanlat                  (tracks, times) float32 29.12 29.19 ... nan nan
    meanlon                  (tracks, times) float32 -95.1 -95.12 ... nan nan
    area                     (tracks, times) float32 137.2 81.5 ... nan nan
    cloudnumber              (tracks, times) float64 8.0 10.0 5.0 ... nan nan
    ...                       ...
    start_split_tracknumber  (tracks) float64 9.781e+03 9.753e+03 ... nan nan
    start_split_timeindex    (tracks) float64 4.0 9.0 14.0 nan ... nan nan nan
    start_split_cloudnumber  (tracks) float64 2.0 3.0 15.0 nan ... nan nan nan
    end_merge_tracknumber    (tracks) float64 9.858e+03 nan ... nan nan
    end_merge_timeindex      (tracks) float64 3.0 nan 16.0 nan ... nan nan nan
    end_merge_cloudnumber    (tracks) float64 10.0 nan 21.0 nan ... nan nan nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

## Extract storms in the region

### Urban area

In [8]:
# Initiation at urban area
def lat_lon_to_cartesian(lat, lon, R=6371):
    x = R * np.radians(lon)
    y = R * np.radians(lat)
    return x, y

start_lon = filtered_ds['cell_meanlon'].isel(times=0)
start_lat = filtered_ds['cell_meanlat'].isel(times=0)
start_hour = filtered_ds['start_basetime'].dt.hour
hour_bin = np.arange(0, 25, 1)

center_lat, center_lon, radius = city_center[1], city_center[0], 76.35
center_x, center_y = lat_lon_to_cartesian(center_lat, center_lon)
storm_x, storm_y = lat_lon_to_cartesian(start_lat, start_lon)
distances = np.sqrt((storm_x - center_x)**2 + (storm_y - center_y)**2)
storms_in_circle = start_hour[distances <= radius]
hist_storms_in_circle, bins = np.histogram(storms_in_circle, bins=hour_bin, range=(0, 24), density=False)
hist_storms_in_circle_LT = np.roll(hist_storms_in_circle, -6)
storms_in_circle
# Fraction of E.SDC tracks to all tracks
#hist_storms_in_circle_LT_frac = 100 * (hist_storms_in_circle_LT / hist_starthour_LT)

<xarray.DataArray 'hour' (tracks: 74)>
array([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19], dtype=int64)
Coordinates:
  * tracks   (tracks) int64 9811 9812 9813 9818 9823 ... 10075 10076 10080 10082
    times    int64 0
Attributes:
    long_name:  Start Epoch time of each track

### Upwind area

In [9]:
storm_points = [Point(lon, lat) for lon, lat in zip(start_lon, start_lat)]
storms_in_upwind_wedge = [start_hour[i] for i, point in enumerate(storm_points) if clipped_upwind_wedge.contains(point)]
storms_in_upwind_wedge = xr.concat(storms_in_upwind_wedge, dim='tracks')
hist_storms_in_upwind_wedge, bins = np.histogram(storms_in_upwind_wedge, bins=hour_bin, range=(0, 24), density=False)
storms_in_upwind_wedge

<xarray.DataArray 'hour' (tracks: 117)>
array([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19],
      dtype=int64)
Coordinates:
  * tracks   (tracks) int64 9810 9815 9817 9820 9822 ... 10079 10081 10084 10085
Attributes:
    long_name:  Start Epoch time of each track

### Downwind area

In [10]:
storm_points = [Point(lon, lat) for lon, lat in zip(start_lon, start_lat)]
storms_in_downwind_wedge = [start_hour[i] for i, point in enumerate(storm_points) if clipped_downwind_wedge.contains(point)]
storms_in_downwind_wedge = xr.concat(storms_in_downwind_wedge, dim='tracks')
hist_storms_in_downwind_wedge, bins = np.histogram(storms_in_downwind_wedge, bins=hour_bin, range=(0, 24), density=False)
storms_in_downwind_wedge

<xarray.DataArray 'hour' (tracks: 2)>
array([15, 17], dtype=int64)
Coordinates:
  * tracks   (tracks) int64 9903 9983
Attributes:
    long_name:  Start Epoch time of each track

In [11]:
circle = storms_in_circle.tracks.values
stats_circle = filtered_ds.sel(tracks=circle)

upwind = storms_in_upwind_wedge.tracks.values
stats_upwind = filtered_ds.sel(tracks=upwind)

downwind = storms_in_downwind_wedge.tracks.values
stats_downwind = filtered_ds.sel(tracks=downwind)

In [12]:
stats_circle

<xarray.Dataset>
Dimensions:                  (tracks: 74, times: 100)
Coordinates:
  * tracks                   (tracks) int64 9811 9812 9813 ... 10076 10080 10082
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) float64 1.0 1.0 6.0 4.0 ... 2.0 2.0 3.0
    base_time                (tracks, times) datetime64[ns] 2021-06-28T14:01:...
    meanlat                  (tracks, times) float32 29.75 nan nan ... nan nan
    meanlon                  (tracks, times) float32 -94.97 nan nan ... nan nan
    area                     (tracks, times) float32 62.75 nan nan ... nan nan
    cloudnumber              (tracks, times) float64 15.0 nan nan ... nan nan
    ...                       ...
    start_split_tracknumber  (tracks) float64 9.753e+03 9.721e+03 ... nan nan
    start_split_timeindex    (tracks) float64 9.0 14.0 nan nan ... nan nan nan
    start_split_cloudnumber  (tracks) float64 3.0 15.0 nan nan ... nan nan nan
    end_merge_tracknumber    (tracks) float64 nan 9.721e+03 nan ... nan nan nan
    end_merge_timeindex      (tracks) float64 nan 16.0 nan nan ... nan nan nan
    end_merge_cloudnumber    (tracks) float64 nan 21.0 nan nan ... nan nan nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

In [13]:
tracks_ds1 = stats_upwind.tracks.values
tracks_ds2 = stats_circle.tracks.values

# Find common tracks
common_tracks = set(tracks_ds1).intersection(tracks_ds2)


### Calculate the cell development stages

In [14]:
# Load the dataset (replace with your dataset loading line if different)
ds = stats_circle

# Extract maxETH_20dbz and calculate the rate of change
maxETH_20dbz = ds['maxETH_20dbz']
rate_of_change = maxETH_20dbz.diff(dim='times')
padded_rate_of_change = xr.concat([xr.full_like(maxETH_20dbz.isel(times=0), np.nan), rate_of_change], dim='times')

# Calculate percentiles for thresholds and classify stages
percentile_25, percentile_75 = np.nanpercentile(padded_rate_of_change, [25, 75])
stages = xr.where(padded_rate_of_change >= percentile_75, 1, np.nan)  # Developing
stages = xr.where((padded_rate_of_change > percentile_25) & (padded_rate_of_change < percentile_75), 2, stages)  # Mature
stages = xr.where(padded_rate_of_change <= percentile_25, 3, stages)  # Dissipating
ds['cell_stages'] = stages.fillna(0)  # Handling initial NaNs

# Final dataset with cloud cell stages classification
final_result = ds


In [15]:
ds

<xarray.Dataset>
Dimensions:                  (tracks: 74, times: 100)
Coordinates:
  * tracks                   (tracks) int64 9811 9812 9813 ... 10076 10080 10082
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/38)
    track_duration           (tracks) float64 1.0 1.0 6.0 4.0 ... 2.0 2.0 3.0
    base_time                (tracks, times) datetime64[ns] 2021-06-28T14:01:...
    meanlat                  (tracks, times) float32 29.75 nan nan ... nan nan
    meanlon                  (tracks, times) float32 -94.97 nan nan ... nan nan
    area                     (tracks, times) float32 62.75 nan nan ... nan nan
    cloudnumber              (tracks, times) float64 15.0 nan nan ... nan nan
    ...                       ...
    start_split_timeindex    (tracks) float64 9.0 14.0 nan nan ... nan nan nan
    start_split_cloudnumber  (tracks) float64 3.0 15.0 nan nan ... nan nan nan
    end_merge_tracknumber    (tracks) float64 nan 9.721e+03 nan ... nan nan nan
    end_merge_timeindex      (tracks) float64 nan 16.0 nan nan ... nan nan nan
    end_merge_cloudnumber    (tracks) float64 nan 21.0 nan nan ... nan nan nan
    cell_stages              (tracks, times) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

In [16]:
ds.maxETH_20dbz[1]

<xarray.DataArray 'maxETH_20dbz' (times: 100)>
array([7.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)
Coordinates:
    tracks   int64 9812
  * times    (times) int64 0 1 2 3 4 5 6 7 8 9 ... 90 91 92 93 94 95 96 97 98 99
Attributes:
    long_name:  Maximum 20dBZ echo-top height in a convective cell
    units:      km

In [17]:
filename = 'C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/precip/Case study/Precip_Jun_28_2021.csv'
pd.read_csv(filename)

,Unnamed: 0,Longitude,Latitude,Precipitation,Date_Time
0,0,-95.264564,29.027106,0.000,6/27/2021 19:00
1,1,-95.226144,29.021331,0.000,6/27/2021 19:00
2,2,-95.187733,29.015533,0.000,6/27/2021 19:00
3,3,-95.149331,29.009713,0.000,6/27/2021 19:00
4,4,-95.110938,29.003871,0.000,6/27/2021 19:00
...,...,...,...,...,...
51241,51241,-94.047113,30.969418,0.125,6/28/2021 20:00
51242,51242,-94.007315,30.962801,0.375,6/28/2021 20:00
51243,51243,-93.967528,30.956161,0.625,6/28/2021 20:00
51244,51244,-93.999580,30.996929,0.750,6/28/2021 20:00
